In [1]:
%cd src

/mnt/e/PhD/Code/Deep-SVDD-PyTorch/src


In [2]:
from datasets.main import load_dataset

In [ ]:
dataset_name = 'mnist'
data_path = '../data' 
normal_class= 3

In [ ]:
dataset_mnist = load_dataset(dataset_name, data_path, normal_class)

In [ ]:
dataset_mnist[0]

In [4]:
import numpy as np

In [ ]:
!pwd

In [10]:
total_data = np.load('/mnt/e/FLARACC/Sukanya_data/data_nor_50.npy')

In [11]:
total_data=total_data[:50]

In [12]:
total_data.shape

(50, 184, 608)

In [13]:
from torch.utils.data import Dataset, DataLoader
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform

    def __len__(self):
        #return len(self.landmarks_frame)
        return total_data.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

#         print(total_data.shape)
#         print(total_data[idx].shape)
        sample=total_data[idx]
        sample = np.array([sample])
        sample = sample.astype('float')
        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
dataset = FaceLandmarksDataset(transform)

In [ ]:
dataloader = DataLoader(dataset, batch_size=10,
                        shuffle=True, num_workers=0)

In [14]:
import torch


In [ ]:
for x in dataloader:
    print(len(x))
    print(x.shape)
    print(x[0].shape)
    print(x[1].shape)
    print(x[2].shape)
    print(x[1])
    break


In [15]:
from datasets.preprocessing import get_target_label_idx, global_contrast_normalization

import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1'))
                               ])
train_set = FaceLandmarksDataset(transform=transform)

In [16]:
dataLoader = DataLoader(train_set, batch_size=10,
                        shuffle=True, num_workers=0)

In [17]:
iterCount = 0
datasetValues = {}

for data in dataLoader:
    if iterCount == 0:
        datasetValues['currDataset'] = data.clone()
    else:
        datasetValues['currDataset'] = torch.cat((datasetValues['currDataset'], data.clone()), 0)

    iterCount += 1


In [18]:
datasetValues['currDataset'].shape

torch.Size([50, 608, 1, 184])

In [20]:
datasetValues['currDataset'].min()

tensor(-8.5016, dtype=torch.float64)

In [21]:
datasetValues['currDataset'].max()

tensor(7.0120, dtype=torch.float64)